# GENERATING IMAGES USING VARIATIONAL AUTOENCODER

## Importing Packages

In [ ]:
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
os.environ["TF_USE_LEGACY_KERAS"] = "1"     # For Keras 2.x compatibility in Keras 3.x installation

## Data Acquisition & Preparation

Uses Fashion MNIST data set containing 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images. Pixel value of the images ranges from 0 through 255.

In [ ]:
# Loads the dataset
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# Normalize the full train set and test set considering the maximum pixel value 
# mentioned in the comment cell above to range values for both the sets between 0 and 1

X_train_full = # ...
X_test = # ...

In [ ]:
# Split the train set further to separate 10000 samples as validation set from full train set stratifically
# by calling method `train_test_split` and passing full train set with labels seperated by comma in
# the first two parameters, then pass 10000 in `test_size` and label `y_train_full` in `stratify` parameter.
# Additionally, an integer can also be passed in parameter.

X_train, X_val, y_train, y_val = # ...

## Modeling

In [ ]:
class Sampling(tf.keras.layers.Layer):
    """
    Custom layer to sample the coding given mean vector (μ) and log variance vector (γ) 
    where γ = log(σ^2). σ is standard deviation vector.
    """
    def call(self, inputs):
        mean, log_var = inputs
        return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + mean

**Modeling Variational Encoder**

In [ ]:
tf.random.set_seed(42)

codings_size = 10   # Length of the mean vector and log variance vector representing latent space of the input data. 

# Use Keras functional APIs as instructed below to create a variational encoder
inputs = # Create an input layer by initializing `tf.keras.layers.Input` with a 
        # `shape` of 2D input image. Shape should a list of sizes.

Z = # Initialize a `tf.keras.layers.Flatten` layer and pass the `inputs` as an argument
    # to pass the images through it to flatten them into 1D.

Z = # Create a dense layer by calling method `tf.keras.layers.Dense` with 150 as first 
    # parameter as number of output units and "relu" to `activation` parameter and then
    # pass previous layer's output `Z` through the layer


Z = # Create a dense layer by calling method `tf.keras.layers.Dense` with 100 as first 
    # parameter as number of output units and "relu" to `activation` parameter and then
    # pass previous layer's output `Z` through the layer


codings_mean = # Create a dense layer by calling method `tf.keras.layers.Dense` with output 
                # unit equal to the coding size as parameter without any activation function 
                # and then pass previous layer's output `Z` through the layer to get mean 
                # vector (μ) out of this layer

codings_log_var = # Create another dense layer exactly similar to the way coding mean layer was 
                    # created in the previous step, but to create it for log variance vector (γ)

codings = # Initialize sampling layer passing a list containing `codings_mean` and `codings_log_var` 
            # as output from the respective layers for samplling layer to sample random coding 
            # vector from Gaussian distribution with with mean μ and standard deviation σ

variational_encoder = # Finally, combine the inputs and outputs by calling method `tf.keras.Model` 
                        # and passing a list containing `inputs` layer to parameter `inputs` and 
                        # another list containing `codings_mean`, `codings_log_var` and `codings` 
                        # to parameter `outputs` to create a variational encoder

**Modeling Variational Decoder**

In [ ]:
decoder_inputs = # Create an input layer by calling `tf.keras.layers.Input` and passing 
                    # a list containing coding size

x = # Create a dense layer calling method `tf.keras.layers.Dense` and passing 100 to first parameter as number of output units
# and "relu" as activation function. Then pass `decoder_inputs` through it.

x = # Create a dense layer calling method `tf.keras.layers.Dense` and passing 150 to first parameter as number of output units
# and "relu" as activation function. Then pass `x` - the previous layer's output, through it.

x = # Create a dense layer calling method `tf.keras.layers.Dense` and passing number of output units equal to the total number 
    # of pixels in 2D input image, and pass `x` - the output of the previous layer, to the initialized layer

outputs = # Create a reshaping layer by calling method `tf.keras.layers.Reshape` passing a list containing shape of the 2D 
            # input image, then pass `x` - the output of the previous layer, to the initialized layer to reshape 1D input into 2D

variational_decoder = # Finally, combine the inputs and outputs by calling method `tf.keras.Model` 
                    # and passing a list containing `decoder_inputs` layer to parameter `inputs` and 
                    # another list containing `outputs` to parameter `outputs` to create a variational decoder

**Combining Encoder & Decoder**

In [ ]:
_, _, codings = # Now, pass `inputs` to the encoder by calling variational encoder as a function and 
                # pass `inputs` as its parameter argument

reconstructions = # Call variational decoder as a function and pass codings received in the previous 
                    # step as its parameter argument to get reconstructed images back

variational_ae = # To combine variational encoder and variational decoder into a variational autoencoder,
                # call `tf.keras.Model` and passing its `inputs` parameter with a list containing `inputs`, and
                # its another parameter `outputs` with a list containing `reconstructions`

In [ ]:
# Latent loss function to push the codings gradually migrate within the coding space
# (also called the latent space) to end up looking like a cloud of Gaussian points.
# It computes the latent loss for each instance in the batch, summing over the last axis.

latent_loss = -0.5 * tf.reduce_sum(
    1 + codings_log_var - tf.exp(codings_log_var) - tf.square(codings_mean),
    axis=-1)

variational_ae.add_loss(
    tf.reduce_mean(latent_loss) / 784.  # Computes the mean loss over all the instances in the batch, 
                                        # followed by dividing the result by 784 to ensure it has the 
                                        # appropriate scale compared to the reconstruction loss.
    )

In [ ]:
# Compiles model
# Call `compile` method of the variational autoencoer mode passing "mse" as arguement to `loss` parameter
# and "nadam" as argument to `optimizer` parameter to compile the model

# Fits the model
# Call `fit` method of the variational autoencoer mode passing the same train set as arguments to its first two parameters,
# and 25 to `epochs`, 128 to `batch_size` and a 2-element tuple containing same validation set to `validation_data`.
# to start model training.
history = # ...

## Generating Images

In [ ]:
# Generates a few random codings and decodes them:

ROWS = 3
COLUMNS = 7

# Samples random codings from a Gaussian distribution and decode them

codings = tf.random.normal(shape=[ROWS * COLUMNS, codings_size])
images = variational_decoder(codings).numpy()

In [ ]:
# Shows some of the fashion items sampled randomly from the generated images

fig,ax = plt.subplots(ROWS, COLUMNS, figsize = (10,5))     # Figure to contain subplots in 5-rows and 9-columns arrangement
ax = ax.ravel()                                 # Flattens the axes allowing accessing each axis contiguously
for i in range(ROWS * COLUMNS):
  ax[i].imshow(images[i], cmap = 'binary')      # Shows the image
  ax[i].axis("off")                             # Set the axis off for being non-relevant in this case

fig.suptitle("Generated Fashion Items")         # Sets title of the figure

## Observations

- Why was Keras functional API used instead of sequential API?

- Why was a custom layer instead of a regular or built-in layer to sample codings? What were its parameters for this layer to generate coding samples? 

- Which technique was followed to ensure coding migrate within the coding space? How did it work?

- Explain in details all the steps followed to generate images and to visualize them.